In [1]:
try:
    import google.colab  # noqa: F401

    %pip install -q dataeval maite-datasets
except Exception:
    pass

In [2]:
import numpy as np
import polars as pl
import torch
from IPython.display import display  # noqa: A004
from maite_datasets.object_detection import VOCDetection
from torchvision.models import ResNet18_Weights, resnet18
from torchvision.transforms.v2 import GaussianNoise

from dataeval import Embeddings, Metadata
from dataeval.core import label_parity
from dataeval.extractors import TorchExtractor
from dataeval.shift import DriftDomainClassifier, DriftKNeighbors, DriftMMD, DriftUnivariate

# Set a random seed
rng = np.random.default_rng(213)

# Set default torch device for notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [3]:
resnet = resnet18(weights=ResNet18_Weights.DEFAULT, progress=False)

# Replace the final fully connected layer with a Linear layer
resnet.fc = torch.nn.Linear(resnet.fc.in_features, 128)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


In [4]:
# Load the training dataset
train_ds = VOCDetection("./data", year="2012", image_set="train", download=True)
print(train_ds)
print(f"Image 0 shape: {train_ds[0][0].shape}")

VOCDetection Dataset
--------------------
    Year: 2012
    Transforms: []
    Image Set: train
    Metadata: {'id': 'VOCDetection_train', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'train'}
    Path: /builds/jatic/aria/dataeval/docs/source/notebooks/data/vocdataset/VOCdevkit/VOC2012
    Size: 5717
Image 0 shape: (3, 442, 500)


In [5]:
# Load the "operational" dataset
operational_ds = VOCDetection("./data", year="2012", image_set="val", download=True)
print(operational_ds)
print(f"Image 0 shape: {train_ds[0][0].shape}")

VOCDetection Dataset
--------------------
    Year: 2012
    Transforms: []
    Image Set: val
    Metadata: {'id': 'VOCDetection_val', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'val'}
    Path: /builds/jatic/aria/dataeval/docs/source/notebooks/data/vocdataset/VOCdevkit/VOC2012
    Size: 5823
Image 0 shape: (3, 442, 500)


In [6]:
# Define pretrained model transformations
transforms = ResNet18_Weights.DEFAULT.transforms()

# Create extractor with model and transforms
extractor = TorchExtractor(resnet, transforms=transforms)

# Create training batches and targets
train_embs = Embeddings(train_ds, extractor=extractor, batch_size=64)

# Create operational batches and targets
operational_embs = Embeddings(operational_ds, extractor=extractor, batch_size=64)

In [7]:
print(f"({len(train_embs)}, {train_embs[0].shape})")  # (5717, shape)
print(f"({len(operational_embs)}, {operational_embs[0].shape})")  # (5823, shape)

(5717, (128,))
(5823, (128,))


In [8]:
# A type alias for all of the drift detectors
DriftDetector = DriftUnivariate | DriftMMD | DriftDomainClassifier | DriftKNeighbors

# Create a mapping for the detectors to iterate over
detectors: dict[str, DriftDetector] = {
    "CVM": DriftUnivariate(method="cvm").fit(train_embs),
    "MMD": DriftMMD().fit(train_embs),
    "MVDC": DriftDomainClassifier().fit(train_embs),
    "KNN": DriftKNeighbors().fit(train_embs),
}

In [9]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(operational_embs).drifted}")

CVM detected drift? False


MMD detected drift? False


MVDC detected drift? False
KNN detected drift? False


In [10]:
# Define transform with added gaussian noise
noisy_transforms = [transforms, GaussianNoise()]

# Create extractor with noisy transforms
noisy_extractor = TorchExtractor(resnet, transforms=noisy_transforms)

# Applies gaussian noise to images before processing
noisy_embs = Embeddings(operational_ds, extractor=noisy_extractor, batch_size=64)

In [11]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(noisy_embs).drifted}")

CVM detected drift? True


MMD detected drift? True


MVDC detected drift? True
KNN detected drift? True


In [12]:
# Store results for inspection
results = {name: detector.predict(noisy_embs) for name, detector in detectors.items()}

In [13]:
cvm_result = results["CVM"]
cvm_details = cvm_result.details

n_drifted = sum(cvm_details["feature_drift"])
n_features = len(cvm_details["feature_drift"])
print(f"Features drifted: {n_drifted}/{n_features}")
print(f"Corrected p-value threshold: {cvm_details['feature_threshold']:.6f}")
print(f"Min feature p-value: {min(cvm_details['p_vals']):.6f}")
print(f"Max feature p-value: {max(cvm_details['p_vals']):.6f}")

Features drifted: 128/128
Corrected p-value threshold: 0.050000
Min feature p-value: 0.000000
Max feature p-value: 0.005169


In [14]:
mvdc_result = results["MVDC"]
mvdc_details = mvdc_result.details

print(f"AUROC: {mvdc_result.distance:.4f} (threshold: {mvdc_result.threshold})")
print(f"Per-fold AUROCs: {[round(a, 4) for a in mvdc_details['fold_aurocs']]}")

# Show top 5 most important features
importances = np.array(mvdc_details["feature_importances"])
top_indices = np.argsort(importances)[::-1][:5]
print("\nTop 5 features driving drift:")
for idx in top_indices:
    print(f"  Feature {idx}: importance = {importances[idx]:.4f}")

AUROC: 0.9978 (threshold: 0.55)
Per-fold AUROCs: [np.float32(0.9984), np.float32(0.9977), np.float32(0.9984), np.float32(0.9981), np.float32(0.9971)]

Top 5 features driving drift:
  Feature 111: importance = 144.0000
  Feature 11: importance = 130.4000
  Feature 121: importance = 122.2000
  Feature 118: importance = 102.8000
  Feature 23: importance = 102.4000


In [15]:
knn_result = results["KNN"]
knn_details = knn_result.details

print(f"Mean reference k-NN distance: {knn_details['mean_ref_distance']:.4f}")
print(f"Mean test k-NN distance:      {knn_details['mean_test_distance']:.4f}")
print(f"Distance increase:             {knn_details['mean_test_distance'] - knn_details['mean_ref_distance']:.4f}")
print(f"P-value: {knn_details['p_val']:.6f}")

Mean reference k-NN distance: 5.0432
Mean test k-NN distance:      5.4583
Distance increase:             0.4151
P-value: 0.000000


In [16]:
mmd_result = results["MMD"]
mmd_details = mmd_result.details

print(f"MMD² distance:   {mmd_result.distance:.6f}")
print(f"MMD² threshold:  {mmd_details['distance_threshold']:.6f}")
print(f"P-value:         {mmd_details['p_val']:.6f}")

MMD² distance:   0.100387
MMD² threshold:  0.000043
P-value:         0.000000


In [17]:
# Build a combined array: first 40% clean, last 60% noisy
n_operational = len(operational_embs)
split_idx = int(n_operational * 0.4)

combined_embs = np.concatenate([operational_embs[:split_idx], noisy_embs[split_idx:]])
print(f"Combined shape: {combined_embs.shape} (clean: {split_idx}, noisy: {n_operational - split_idx})")

Combined shape: (5823, 128) (clean: 2329, noisy: 3494)


In [18]:
# Re-fit detectors with chunking enabled (5 chunks each)
chunked_detectors: dict[str, DriftDetector] = {
    "CVM": DriftUnivariate(method="cvm").fit(train_embs, chunk_count=5),
    "MMD": DriftMMD().fit(train_embs, chunk_count=5),
    "MVDC": DriftDomainClassifier(threshold=(0.45, 0.65)).fit(train_embs, chunk_count=5),
    "KNN": DriftKNeighbors().fit(train_embs, chunk_count=5),
}

In [19]:
for name, detector in chunked_detectors.items():
    result = detector.predict(combined_embs)
    print(f"\n{name} - Overall drift detected: {result.drifted} (metric: {result.metric_name})")
    if isinstance(result.details, pl.DataFrame):
        display(result.details)


CVM - Overall drift detected: True (metric: cvm_distance)


key,index,start_index,end_index,value,upper_threshold,lower_threshold,drifted
str,i64,i64,i64,f64,f64,f64,bool
"""[0:1143]""",0,0,1143,1.622,2.195479,0.0,false
"""[1144:2287]""",1,1144,2287,0.205438,2.195479,0.0,false
"""[2288:3431]""",2,2288,3431,20.7708,2.195479,0.0,true
"""[3432:4575]""",3,3432,4575,22.211973,2.195479,0.0,true
"""[4576:5822]""",4,4576,5822,23.342222,2.195479,0.0,true



MMD - Overall drift detected: True (metric: mmd2)


key,index,start_index,end_index,value,upper_threshold,lower_threshold,drifted
str,i64,i64,i64,f64,f64,f64,bool
"""[0:1143]""",0,0,1143,0.006854,0.009899,-0.004838,false
"""[1144:2287]""",1,1144,2287,0.000168,0.009899,-0.004838,false
"""[2288:3431]""",2,2288,3431,0.096612,0.009899,-0.004838,true
"""[3432:4575]""",3,3432,4575,0.102833,0.009899,-0.004838,true
"""[4576:5822]""",4,4576,5822,0.100748,0.009899,-0.004838,true



MVDC - Overall drift detected: True (metric: auroc)


key,index,start_index,end_index,value,upper_threshold,lower_threshold,drifted
str,i64,i64,i64,f64,f64,f64,bool
"""[0:1143]""",0,0,1143,0.617572,0.65,0.45,false
"""[1144:2287]""",1,1144,2287,0.494803,0.65,0.45,false
"""[2288:3431]""",2,2288,3431,0.970899,0.65,0.45,true
"""[3432:4575]""",3,3432,4575,0.996776,0.65,0.45,true
"""[4576:5822]""",4,4576,5822,0.9965,0.65,0.45,true



KNN - Overall drift detected: True (metric: knn_distance)


key,index,start_index,end_index,value,upper_threshold,lower_threshold,drifted
str,i64,i64,i64,f64,f64,f64,bool
"""[0:1143]""",0,0,1143,4.975564,5.171253,4.915223,false
"""[1144:2287]""",1,1144,2287,5.080391,5.171253,4.915223,false
"""[2288:3431]""",2,2288,3431,5.460795,5.171253,4.915223,true
"""[3432:4575]""",3,3432,4575,5.515712,5.171253,4.915223,true
"""[4576:5822]""",4,4576,5822,5.456972,5.171253,4.915223,true


In [20]:
# Get the metadata for each dataset
train_md = Metadata(train_ds)
operational_md = Metadata(operational_ds)

# The VOC dataset has 20 classes
label_parity(train_md.class_labels, operational_md.class_labels, num_classes=20)["p_value"]

0.949856067521638